In [1]:
import os
import sys
import pickle
from pathlib import Path

import cv2
import numpy as np
import pandas as ps
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from fastprogress import progress_bar

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

sys.path.append("../src")

from metrics import gap
from models import EncoderWithHead
from models.efficientnets import EfficientNetEncoder
from models.heads import CosFace
from batteries.progress import tqdm
from batteries import t2d, load_checkpoint

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

print("CUDA is available!" if torch.cuda.is_available() else "Something went wrong - there is no CUDA!")

CUDA is available!


In [3]:
class TestDataset(Dataset):
    def __init__(self, images, transforms=None):
        self.images = images
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, idx: int):
        file = str(self.images[idx])
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        return img

In [4]:
IMAGES_DIR = Path("..") / "input" / "train"
train_df = ps.read_pickle("../input/train_valid.pkl")

train_df.head()

,id,landmark_id,is_valid
0,17660ef415d37059,1,False
1,92b6290d571448f6,1,True
2,cd41bf948edc0340,1,False
3,fb09f1e98c6d2f70,1,False
4,25c9dfc7ea69838d,7,False


In [5]:
def id_to_file(id_: str) -> Path:
    return IMAGES_DIR / id_[0] / id_[1] / id_[2] / f"{id_}.jpg"


inference_set = TestDataset(
    [id_to_file(file) for file in train_df["id"].values],
    transforms=albu.Compose([
        albu.Resize(224, 224),
        albu.Normalize(),
        ToTensorV2(),
    ])
)
inference_loader = DataLoader(
    inference_set,
    batch_size=128,
    num_workers=16,
    shuffle=False
)

len(inference_set), len(inference_loader)

(1580470, 12348)

In [6]:
EMBEDDING_SIZE = 512
NUM_CLASSESS = 81313

device = torch.device("cuda:0")

model = EncoderWithHead(
    EfficientNetEncoder("efficientnet-b0", EMBEDDING_SIZE, bias=False),
    CosFace(EMBEDDING_SIZE, NUM_CLASSESS, None),
)
load_checkpoint("../logs/full_set2/stage_0/last.pth", model)

model = model.encoder
model = model.to(device)
model = model.eval()
# model

Loaded pretrained weights for efficientnet-b0
<= Loaded model from '../logs/full_set2/stage_0/last.pth'


In [7]:
def get_embedding(model, loader, device):
    model.eval()
    embedding = []
    with torch.no_grad():
        for images in progress_bar(loader):
            images = images.to(device)
            output = F.normalize(model(images)).detach().cpu().numpy()
            embedding.append(output)
    return np.concatenate(embedding, axis=0)

In [8]:
train_ids = train_df["id"].values
train_emb = get_embedding(model, inference_loader, device)

KeyboardInterrupt: 